In [ ]:
!pip install -q -U transformers accelerate peft datasets bitsandbytes flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 77.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
  

In [ ]:

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)

In [ ]:
from peft import (
    get_peft_model,
    PromptTuningConfig,
    PromptTuningInit,
    TaskType
)

In [ ]:
from datasets import Dataset
import torch
import json
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/sampled_6000.json'
drive_path = "/content/drive/MyDrive/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if not os.path.exists(file_path):
    print(f"Error: File not found at {file_path}")
    print("Available files in directory:")
    print(os.listdir(drive_path))
else:
    print(f"Found file at {file_path}")

Found file at /content/drive/MyDrive/sampled_6000.json


In [ ]:
import json

try:
    with open(file_path) as f:
        try:
            medical_data = json.load(f)
        except json.JSONDecodeError:

            content = f.read().split('[file content end]')[0].split('[file content begin]')[-1].strip()
            medical_data = json.loads(content)

    print(f"Successfully loaded {len(medical_data)} medical examples")


except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {str(e)}")

Successfully loaded 6000 medical examples


In [ ]:

print("\nSample data item:")
print(f"Instruction: {medical_data[0]['instruction']}")
print(f"Input: {medical_data[0]['input'][:50]}...")
print(f"Output: {medical_data[0]['output'][:50]}...")


Sample data item:
Instruction: If you are a doctor, please answer the medical questions based on the patient's description.
Input: I wake in the night, usually about 2-3 hours after...
Output: Dear patient Here are the possibilities of what yo...


In [ ]:
def format_data(sample):
        return {
            "text": f"<s>[MED] {sample['instruction']}\nPatient: {sample['input']}\nDoctor: {sample['output']}</s>"
        }

In [ ]:
dataset = [format_data(d) for d in medical_data]
dataset = Dataset.from_list(dataset)

In [ ]:
drive.mount('/content/drive')
output_dir = "/content/drive/MyDrive/medical_prompt_tuning"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def find_latest_checkpoint(output_dir):
    try:

        if not os.path.exists(output_dir):
            print(f"Output directory {output_dir} does not exist")
            return None


        if not os.listdir(output_dir):
            print(f"Output directory {output_dir} is empty")
            return None


        checkpoints = [d for d in os.listdir(output_dir)
                      if d.startswith("checkpoint") and os.path.isdir(os.path.join(output_dir, d))]

        if not checkpoints:
            print("No checkpoint directories found")
            return None


        checkpoints.sort(key=lambda x: int(x.split("-")[1]))
        latest = os.path.join(output_dir, checkpoints[-1])
        print(f"Found checkpoint: {latest}")
        return latest

    except Exception as e:
        print(f"Error finding checkpoint: {e}")
        return None

In [ ]:
latest_checkpoint = find_latest_checkpoint(output_dir)
print(f"Latest checkpoint: {latest_checkpoint}")

Found checkpoint: /content/drive/MyDrive/medical_prompt_tuning/checkpoint-1450
Latest checkpoint: /content/drive/MyDrive/medical_prompt_tuning/checkpoint-1450


In [ ]:
from peft import PeftModel, PeftConfig
config_path= "/content/drive/MyDrive/medical_prompt_tuning/checkpoint-1450"
config = PeftConfig.from_pretrained(config_path)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(latest_checkpoint)

In [ ]:

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
mistral = PeftModel.from_pretrained(
    model,
    latest_checkpoint,
    device_map="auto"
)

In [ ]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 81,920 || all params: 7,248,105,472 || trainable%: 0.0011


In [ ]:
def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",
            truncation=True,
            max_length=256,
            return_tensors=None
        )

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:

tokenized_dataset = tokenized_dataset.map(
        lambda examples: {"labels": examples["input_ids"].copy()},
        batched=True
    )


Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

In [ ]:

training_args = TrainingArguments(
    output_dir=os.path.join(drive_path, "medical2_prompt_tuning"),
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=3e-2,
    weight_decay=0.01,
    logging_steps=20,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none",
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    dataloader_num_workers=2,
    remove_unused_columns=True,
    max_steps=1900,
    label_names=["input_ids", "attention_mask", "labels"]
)


In [ ]:
trainer = Trainer(
    model=mistral,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

In [ ]:
trainer.train(resume_from_checkpoint=latest_checkpoint)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1460,2.052000
1480,2.042200
1500,2.007600
1520,2.107600
1540,2.066900
1560,2.035100
1580,2.013500
1600,2.045500
1620,2.082100
1640,2.057700


KeyboardInterrupt: 

In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/medical_prompt_tuning/tokenizer_config.json',
 '/content/drive/MyDrive/medical_prompt_tuning/special_tokens_map.json',
 '/content/drive/MyDrive/medical_prompt_tuning/chat_template.jinja',
 '/content/drive/MyDrive/medical_prompt_tuning/tokenizer.model',
 '/content/drive/MyDrive/medical_prompt_tuning/added_tokens.json',
 '/content/drive/MyDrive/medical_prompt_tuning/tokenizer.json')

In [ ]:
output_dir = "/content/drive/MyDrive/medical2_prompt_tuning"

In [ ]:
def find_latest_checkpoint(output_dir):
    try:

        if not os.path.exists(output_dir):
            print(f"Output directory {output_dir} does not exist")
            return None


        if not os.listdir(output_dir):
            print(f"Output directory {output_dir} is empty")
            return None


        checkpoints = [d for d in os.listdir(output_dir)
                      if d.startswith("checkpoint") and os.path.isdir(os.path.join(output_dir, d))]

        if not checkpoints:
            print("No checkpoint directories found")
            return None


        checkpoints.sort(key=lambda x: int(x.split("-")[1]))
        latest = os.path.join(output_dir, checkpoints[-1])
        print(f"Found checkpoint: {latest}")
        return latest

    except Exception as e:
        print(f"Error finding checkpoint: {e}")
        return None

In [ ]:
def medical_response(patient_input):
    prompt = f"""<|user|>
As a doctor, answer medical questions
Patient: {patient_input}<|end|>
<|assistant|>
"""

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens = 256,
        temperature = 0.7,
        pad_token_id = tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip()


In [ ]:
latest_checkpoint = find_latest_checkpoint(output_dir)
print(f"Latest checkpoint: {latest_checkpoint}")

Found checkpoint: /content/drive/MyDrive/medical2_prompt_tuning/checkpoint-1700
Latest checkpoint: /content/drive/MyDrive/medical2_prompt_tuning/checkpoint-1700


In [ ]:
print(medical_response("I have fever and body aches"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


I'm not a doctor, but I can try to help you understand your symptoms. Fever and body aches can be signs of various conditions, such as the flu, a cold, or even COVID-19. However, it's important to consult a healthcare professional for an accurate diagnosis and treatment. In the meantime, you can try to manage your symptoms with over-the-counter medications like ibuprofen or acetaminophen. Stay hydrated and rest as much as possible. If your symptoms worsen or you have difficulty breathing, seek immediate medical attention.


In [ ]:
print(medical_response("A 45-year-old male presents with fatigue, weight gain, and cold intolerance. What are the possible differential diagnoses, and which lab tests would you recommend?"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Given the symptoms of fatigue, weight gain, and cold intolerance, some possible differential diagnoses include:

1. Hypothyroidism: This condition is characterized by low thyroid hormone levels, which can lead to weight gain, fatigue, and cold intolerance.

2. Diabetes Mellitus: Symptoms of diabetes include increased thirst, frequent urination, and weight gain. Fatigue can also be a symptom of uncontrolled diabetes.

3. Cushing's Syndrome: This condition is caused by an overproduction of cortisol, a hormone produced by the adrenal glands. Symptoms can include weight gain, fatigue, and cold intolerance.

4. Chronic Fatigue Syndrome: This is a complex disorder characterized by extreme fatigue that doesn't go away with rest. Other symptoms can include weakness, muscle pain, and difficulty thinking or concentrating.

5. Depression: Depression can cause symptoms such as fatigue, weight gain, and decreased interest in activities.

For lab tests, I would recommend:

1. Thyroid function tests 

In [ ]:
print(medical_response("What are the key clinical features that differentiate bacterial meningitis from viral meningitis"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Bacterial meningitis and viral meningitis share some similar symptoms, but there are key differences that can help in differentiating between the two. Here are some of the key clinical features:

1. Speed of onset: Bacterial meningitis tends to develop more rapidly, often within hours or days, while viral meningitis usually develops over a period of several days.

2. Fever: Both conditions can cause fever, but fever is often higher and more severe in bacterial meningitis.

3. Headache: Both conditions can cause severe headache, but the headache in bacterial meningitis is often more intense and persistent.

4. Neck stiffness: Both conditions can cause neck stiffness, but the neck stiffness in bacterial meningitis is often more pronounced.

5. Altered mental status: Both conditions can cause altered mental status, but the alteration in bacterial meningitis is often more severe and can include confusion, delirium, or coma.

6. Vomiting: Both conditions can cause vomiting, but vomiting is 

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, notebook_login
repo_name = "gabbar427/mediguide"
create_repo(repo_name, repo_type="model", exist_ok=True)

RepoUrl('https://huggingface.co/gabbar427/mediguide', endpoint='https://huggingface.co', repo_type='model', repo_id='gabbar427/mediguide')

In [ ]:
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Uploading...:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Uploading...:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gabbar427/mediguide/commit/3023c574f71c538b7577062370404577b33bef30', commit_message='Upload tokenizer', commit_description='', oid='3023c574f71c538b7577062370404577b33bef30', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gabbar427/mediguide', endpoint='https://huggingface.co', repo_type='model', repo_id='gabbar427/mediguide'), pr_revision=None, pr_num=None)

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("Mohammed-Altaf/medical-instruction-120k")
test_split = raw_datasets["test"]

import re
def extract_prompt_response(example):
    """
    Parses the single-string conversation field into:
      - instruction: text between “[|Human|]” and “[|AI|]” (or full text if no markers).
      - response: text after the last “[|AI|]” marker (or empty if none).
    """
    convo = str(example[next(iter(example.keys()))]).strip()
    human_match = re.search(r"\[\|Human\|\]\s*(.*?)\s*(?=\[\|AI\|\])", convo, re.DOTALL)
    instruction = human_match.group(1).strip() if human_match else convo
    parts = re.split(r"\[\|AI\|\]", convo)
    response = parts[-1].strip() if len(parts) > 1 else ""
    return {"instruction": instruction, "response": response}

test_df = test_split.map(
    extract_prompt_response,
    remove_columns=test_split.column_names,
    num_proc=4
)
test_prompts = test_df["instruction"]
test_references = test_df["response"]

README.md:   0%|          | 0.00/965 [00:00<?, ?B/s]

medicare_110k_train.json:   0%|          | 0.00/126M [00:00<?, ?B/s]

medicare_110k_test.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/106556 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5609 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5609 [00:00<?, ? examples/s]

In [ ]:
test_prompts[0]

'I wake in the night, usually about 2-3 hours after going to sleep, with both feet and legs to mid calf feeling like they are on fire. slight red discolorization, minor swelling. This is very painful but after getting up, I can walk it off in about 30 minutes.'

In [ ]:
test_references[0]

'Dear patient Here are the possibilities of what you might have.1)PhlebitisPhlebitis means inflammation of the veins, and can cause redness, itching, irritation, pain, and swelling. A simple Doppler can rule this out.2Blood clot in the lifeblood clots in the leg can become very dangerous, symptoms include swelling, redness, tenderness in the leg. Coagulation profile with an angiography may be required3)Cellulitis: Initial stage. Only can be clinically ruled out Hope this helped'

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling

# ——————————————————————————————————————
#  Compute Test Perplexity with Resume Capability
# ——————————————————————————————————————


checkpoint_dir = "/content/drive/MyDrive/medical2_prompt_tuning"
eval_ckpt_path = "/content/drive/MyDrive/medical2_prompt_tuning/test_eval_state.pth"
batch_size = 2
save_every_n_batches = 50


class LMTestDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=1024):
        encodings = tokenizer(
            texts,
            return_tensors="pt",
            max_length=max_length,
            truncation=True,
            padding="max_length"
        )
        self.input_ids = encodings["input_ids"]
        self.attention_mask = encodings["attention_mask"]

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.input_ids[idx].clone()
        }


test_texts = [
    f"{instr}\n\n{resp}"
    for instr, resp in zip(test_prompts, test_references)
]
lm_test_dataset = LMTestDataset(test_texts, tokenizer, max_length=1024)


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)
test_loader = DataLoader(
    lm_test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator
)

device = mistral.device


if os.path.isfile(eval_ckpt_path):

    state = torch.load(eval_ckpt_path)
    start_batch = state["last_batch"] + 1
    accumulated_loss = state["accumulated_loss"]
    total_tokens = state["total_tokens"]
    print(f"Resuming test-eval from batch {start_batch} (saved on disk).")
else:

    start_batch = 0
    accumulated_loss = 0.0
    total_tokens = 0
    print("Starting test-eval from batch 0.")


mistral.eval()
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        if batch_idx < start_batch:
            continue


        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)


        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss.detach().cpu().item()


        nonpad_tokens = (labels != tokenizer.pad_token_id).sum().item()


        accumulated_loss += loss * nonpad_tokens
        total_tokens += nonpad_tokens


        if (batch_idx + 1) % save_every_n_batches == 0:
            state = {
                "last_batch": batch_idx,
                "accumulated_loss": accumulated_loss,
                "total_tokens": total_tokens
            }
            torch.save(state, eval_ckpt_path)
            print(f" Saved eval state at batch {batch_idx} → tokens={total_tokens}")

    final_avg_loss = accumulated_loss / total_tokens
    test_perplexity = torch.exp(torch.tensor(final_avg_loss)).item()
    os.remove(eval_ckpt_path)

    print(f"\n→ Test complete. Avg. token-loss = {final_avg_loss:.4f}")
    print(f"→ Test Perplexity = {test_perplexity:.2f}")


Starting test-eval from batch 0.
 Saved eval state at batch 49 → tokens=102400
 Saved eval state at batch 99 → tokens=204800
 Saved eval state at batch 149 → tokens=307200
 Saved eval state at batch 199 → tokens=409600
 Saved eval state at batch 249 → tokens=512000
 Saved eval state at batch 299 → tokens=614400
 Saved eval state at batch 349 → tokens=716800
 Saved eval state at batch 399 → tokens=819200
 Saved eval state at batch 449 → tokens=921600
 Saved eval state at batch 499 → tokens=1024000
 Saved eval state at batch 549 → tokens=1126400
 Saved eval state at batch 599 → tokens=1228800
 Saved eval state at batch 649 → tokens=1331200
 Saved eval state at batch 699 → tokens=1433600
 Saved eval state at batch 749 → tokens=1536000
 Saved eval state at batch 799 → tokens=1638400
 Saved eval state at batch 849 → tokens=1740800
 Saved eval state at batch 899 → tokens=1843200
 Saved eval state at batch 949 → tokens=1945600
 Saved eval state at batch 999 → tokens=2048000
 Saved eval state 

In [ ]:
import time
import torch

# ——————————————————————————————————————
#  Measure Average Latency (Generation Time)
# ——————————————————————————————————————


n_samples = min(50, len(test_prompts))
max_new_tokens = 128
batch_size = 4


tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

latencies = []
model.eval()
device = model.device


dummy_input = tokenizer("Hello", return_tensors="pt").to(device)
_ = model.generate(
    **dummy_input,
    max_new_tokens=10,
    do_sample=False,
    use_cache=True,
    return_dict_in_generate=False
)
torch.cuda.synchronize()


for i in range(0, n_samples, batch_size):
    batch_prompts = test_prompts[i : i + batch_size]
    inputs = tokenizer(
        batch_prompts,
        return_tensors="pt",
        truncation=True,
        padding=True
    ).to(device)

    torch.cuda.synchronize()
    start = time.time()
    _ = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        use_cache=True,
        return_dict_in_generate=False
    )
    torch.cuda.synchronize()
    end = time.time()


    elapsed = end - start
    latencies.append(elapsed / len(batch_prompts))

avg_latency = sum(latencies) / len(latencies)
print(f"Average Latency (per prompt, {max_new_tokens} new tokens): {avg_latency:.4f} seconds")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 

Average Latency (per prompt, 128 new tokens): 7.4016 seconds


In [ ]:
import os

# ——————————————————————————————————————
#  Compute Model Size on Disk
# ——————————————————————————————————————

def folder_size_in_mb(path: str) -> float:
    total_bytes = 0
    for root, _, files in os.walk(path):
        for fname in files:
            fp = os.path.join(root, fname)
            total_bytes += os.path.getsize(fp)
    return total_bytes / (1024 ** 2)



model_size_mb = folder_size_in_mb(latest_checkpoint)
print(f"Model Size on Disk (under “{latest_checkpoint}”): {model_size_mb:.1f} MB")

Model Size on Disk (under “/content/drive/MyDrive/medical2_prompt_tuning/checkpoint-1750”): 5.2 MB


In [ ]:
# ——————————————————————————————————————
# ROUGE Generation with Resume Capability
# ——————————————————————————————————————

import os
import torch
import json
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
def optimized_medical_generation(model, tokenizer, prompts, references,
                                 checkpoint_path, batch_size=8, max_length=512):
    """FIXED: Efficient generation with resume capability"""
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            state = json.load(f)
        completed_indices = state.get('completed_indices', [])
        predictions = state.get('predictions', [])
        print(f"Resuming from {len(completed_indices)} completed samples")
    else:
        completed_indices = []
        predictions = []
        state = {'completed_indices': completed_indices, 'predictions': predictions}


    completed_set = set(completed_indices)


    remaining_indices = [i for i in range(len(prompts)) if i not in completed_set]

    if not remaining_indices:
        print("All samples already processed!")
        return predictions, [references[i] for i in completed_indices]

    model.eval()
    device = model.device


    for batch_start in tqdm(range(0, len(remaining_indices), batch_size), desc="Medical Generation"):
        batch_end = min(batch_start + batch_size, len(remaining_indices))
        batch_indices = remaining_indices[batch_start:batch_end]
        batch_prompts = [prompts[i] for i in batch_indices]


        inputs = tokenizer(
            [f"MEDICAL PROMPT: {p}" for p in batch_prompts],  R FORMAT
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        ).to(device)

        with torch.no_grad():

            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                pad_token_id=tokenizer.pad_token_id,
                do_sample=False,
                num_beams=1,
                use_cache=True,
            )


        decoded = tokenizer.batch_decode(
            outputs,
            skip_special_tokens=True
        )


        batch_preds = []
        for idx, (prompt, text) in enumerate(zip(batch_prompts, decoded)):

            if text.startswith(f"MEDICAL PROMPT: {prompt}"):
                gen_text = text[len(f"MEDICAL PROMPT: {prompt}"):].strip()
            else:
                gen_text = text.strip()
            batch_preds.append(gen_text)


        predictions.extend(batch_preds)
        completed_indices.extend(batch_indices)


        with open(checkpoint_path, 'w') as f:
            json.dump({
                'completed_indices': completed_indices,
                'predictions': predictions
            }, f)

    return predictions, [references[i] for i in completed_indices]

In [ ]:
checkpoint_path = "/content/drive/MyDrive/rouge_checkpoint.json"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(latest_checkpoint)

In [ ]:
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

In [ ]:

model_name = "gabbar427/mediguide"



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
predictions, processed_refs = optimized_medical_generation(
        model, tokenizer, test_prompts, test_references, checkpoint_path,
        batch_size=8,
        max_length=384
    )

Resuming from 2824 completed samples


Medical Generation: 100%|██████████| 22/22 [21:55<00:00, 59.80s/it]


In [ ]:
from evaluate import load
rouge = load("rouge")
results = rouge.compute(
        predictions=predictions,
        references=processed_refs,
        use_stemmer=True,
        use_aggregator=True
    )

In [ ]:

print("\nMedical ROUGE Scores:")
print(f"ROUGE-1: {results['rouge1']:.4f}")
print(f"ROUGE-2: {results['rouge2']:.4f}")
print(f"ROUGE-L: {results['rougeL']:.4f}")
print(f"ROUGE-Lsum: {results['rougeLsum']:.4f}")


Medical ROUGE Scores:
ROUGE-1: 0.2593
ROUGE-2: 0.0342
ROUGE-L: 0.1310
ROUGE-Lsum: 0.1595


In [ ]:
with open("/content/drive/MyDrive/medical_rouge_results.json", "w") as f:
        json.dump({
            "scores": results,
            "predictions": predictions,
            "references": processed_refs
        }, f)